In [1]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import ee
from unidecode import unidecode
import os


In [4]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6mN9HWaOOG_9j0SZ9KJZnMZVAF81Clcrbzcu3Sz3tbM&tc=myMVDaokqFsGG9r8HewoVPALXGGrSeiepuTZ9eeG6iQ&cc=CO5kSCIuu6nVJZsMd2mQ2ckfYVRZYN6NsViXXqsdGvw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqWfGdFigUd3mQwYE7-2rTt43kzy9RBZKF6WxrXs-i-AZFBASnGHwU

Successfully saved authorization token.


In [5]:
gdrive_folder = 'Italy land cover data'


In [7]:
italy = 'users/basanthqo/Italy_prov'
county_region = ee.FeatureCollection(italy)
feature_list = county_region.toList(1e5)
feature_list_computed = feature_list.getInfo()


In [8]:
CLEAN_NAME = lambda r, l: r.get('properties').get(l)

In [9]:
def ftr_key_fn(region):
 return f"DEN_PROV_{CLEAN_NAME(region, 'DEN_PROV')}-COD_REG_{CLEAN_NAME(region,'COD_REG')}-COD_PROV_{CLEAN_NAME(region, 'COD_PROV')}"


In [10]:
# image collection bands
agri_lc_band = ['crops-coverfraction']

In [ ]:
# Import the copernicus land cover collection.
agri_lc = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")

#MOD13A1.006 Terra Vegetation Indices 16-Day Global 500m
# veg_ind= ee.ImageCollection("MODIS/006/MOD13A1")

In [12]:
bands = agri_lc_band

In [13]:
def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select(bands)
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

In [14]:
def export_to_cloud(img, fname, folder, expregion, eeuser=None, scale=500):
  # print "export to cloud"
  expcoord = expregion.geometry().coordinates().getInfo()[0]
  expconfig = dict(description=fname, folder=folder, fileNamePrefix=fname, dimensions=None, region=expcoord,
                   scale=scale, crs='EPSG:4326', crsTransform=None, maxPixels=1e13)
  task = ee.batch.Export.image.toDrive(image=img.clip(expregion), **expconfig)
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    time.sleep(10)
  print ('Done.', task.status())

In [17]:
start_date ='2016-01-1'
end_date = '2022-09-30'

In [18]:
imgcoll = ee.ImageCollection(agri_lc).filterDate(start_date,end_date)
img=imgcoll.iterate(appendBand)
img=ee.Image(img)

In [19]:
keys_with_issues = []
count_already_downloaded = 0
count_filtered = 0

for idx, region in enumerate(feature_list_computed):
  if not True:
      count_filtered += 1
      continue
      
  subunit_key = ftr_key_fn(region)
  file_name = f'Italy_land_cover{subunit_key}_{start_date}_{end_date}'
  print(file_name)
  if os.path.isfile(os.path.join(file_name + '.tif')):
          print (subunit_key, 'already downloaded. Continuing...')
          count_already_downloaded += 1
          continue

  try:
      export_to_cloud(img, file_name, gdrive_folder, ee.Feature(region), scale=500)
  except KeyboardInterrupt:
      print ('received SIGINT, terminating execution')
      break
  except Exception as e:
    print ('issue with {} ({})'.format(subunit_key, str(e)))

Italy_land_coverDEN_PROV_Vercelli-COD_REG_1-COD_PROV_2_2016-01-1_2022-09-30
Done. {'state': 'READY', 'description': 'Italy_land_coverDEN_PROV_Vercelli-COD_REG_1-COD_PROV_2_2016-01-1_2022-09-30', 'creation_timestamp_ms': 1665832673176, 'update_timestamp_ms': 1665832673176, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '3JVG4FIOJVTHMPGYCJ3CPIPV', 'name': 'projects/earthengine-legacy/operations/3JVG4FIOJVTHMPGYCJ3CPIPV'}
Italy_land_coverDEN_PROV_Novara-COD_REG_1-COD_PROV_3_2016-01-1_2022-09-30
Done. {'state': 'READY', 'description': 'Italy_land_coverDEN_PROV_Novara-COD_REG_1-COD_PROV_3_2016-01-1_2022-09-30', 'creation_timestamp_ms': 1665832674927, 'update_timestamp_ms': 1665832674927, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '34T677E3BDWF4JU7M75HT2T4', 'name': 'projects/earthengine-legacy/operations/34T677E3BDWF4JU7M75HT2T4'}
Italy_land_coverDEN_PROV_Cuneo-COD_REG_1-COD_PROV_4_2016-01-1_2022-09-30
Done. {'state': 'READY', 'description': 'Italy_land_coverDE